# 🧑‍💻 Amazon Nova built-in System Tools: Code Interpreter & Web Grounding

Amazon Nova 2 comes with two built-in system tools:
1. Code Interpreter allows Amazon Nova to execute code
2. Web Grounding enables Amazon Nova to search for real-time information on the web

The Code Interpreter allows Amazon Nova to generate code that get's executed in a secure sandbox. 
This is useful for problems that require math, code writing or code review, and data analysis.

Example Use Cases:
- Data Analysis, Data Visualization, Data-Driven Decision-Making
- Code Assistant, Code Reviewer
- Math
- Other use cases that require reproducible computation rather than probablistic next token prediction

When not to use Code Interpreter:
- If you need to retrieve real-time information from the web. For such use cases we recommend to use the Amazon Nova web search system tool.
- Also currently, Amazon Nova Code Interpreter does not have access to your AWS resources. So for automation tasks or to get and upload data to an Amazon S3 bucket or database we recommend to user Amazon Bedrock AgentCore.

This notebook guides you how to use the Code Interpreter.

## 🎯 What You'll Build

By the end of this workshop, you will:
- ✅ Understand how the API for Code Interpreter & Web Grounding works
- ✅ Know how to use Code Interpreter & Web Grounding system tools
- ✅ Best practices for using system tools

## Architecture of Code Interpreter

![](./img/Code-Interpreter.png)

## Prerequisites
- AWS Account
- [IAM Permissions](https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html) or an Amazon Bedrock [API Key](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_bedrock.html) to invoke Amazon Nova 2
- Python 3.8+

## Workshop Outline
1. Setup and Configuration
2. Quick start with Nova Code Interpreter
3. Sample Use Cases that require Code execution
4. Best Practices for Code Intepreter
5. Next Steps

## 1. Setup and Configuration

This section covers the initial setup required to use Amazon Nova's Code Interpreter.
We'll configure the AWS SDK (boto3) with appropriate settings for code execution tasks.

In [1]:
%pip install --quiet -r requirementsv2.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.40.26 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.1 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.1 which is incompatible.
sagemaker-studio-analytics-extension 0.2.2 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Before running this script, ensure you have the required dependencies installed:

In [2]:
#%pip install --quiet "boto3>=x.x.x." "botocore>=x.x.x." TODO after release

In [2]:
from IPython.display import Markdown, JSON

import boto3
from botocore.config import Config
import json 

The `create_boto3_client` helper function initializes a boto3 client for Amazon Bedrock Runtime
with custom timeout configurations. These extended timeouts are crucial for Code Interpreter
tasks that may take longer to execute.

In [3]:
def create_boto3_client():
    client = boto3.client(
        "bedrock-runtime",
        region_name="us-west-2",
        config=Config(
            connect_timeout=3600,  # 60 minutes code interpreter might be more long running
            read_timeout=3600,     # 60 minutes
            retries={'max_attempts': 1}
        )
    )
    return client

In [4]:
bedrock_client = create_boto3_client()

In [5]:
NOVA_2_LITE = "us.amazon.nova-2-lite-omni-v1:0"
NOVA_2_LITE = "us.amazon.nova-2-omni-v1:0"

## 2. Quick start with System Tools

### Converse API

This section demonstrates how to use Amazon Nova's Code Interpreter or Web Grounding through the Converse API.
The Converse API provides a unified interface for interacting with foundation models,
including support for system tools like the Code Interpreter and Web Grounding.

The `construct_converse_payload` helper function constructs the payload required for the Converse API.
It includes the model ID, messages, and other parameters like temperature, max tokens, and tool configurations.

In [6]:
def construct_converse_payload(model, messages, temperature, max_tokens, top_p = 0.9, system = None, reasoning_effort = None, tools = None):
    inferenceConfig = {
            "maxTokens": max_tokens,
    }

    if temperature:
        inferenceConfig["temperature"] = temperature

    if top_p:
        inferenceConfig["topP"] = top_p

    tool_config = None
    if tools:
        tool_config = {
            "tools": tools
          }

    additional_request_fields = {}
    if reasoning_effort:
        reasoning_config = {
          "type": "enabled",
          "maxReasoningEffort": reasoning_effort
        }
        additional_request_fields["reasoningConfig"] = reasoning_config

    request = {
        "modelId":model, 
        "messages":messages, 
        "inferenceConfig":inferenceConfig,
        "toolConfig":tool_config,
        "additionalModelRequestFields":additional_request_fields
    }

    if system:
        request["system"] = system
    
    return request

The `amazon_bedrock_nova_converse` helper function is a wrapper that:
1. Constructs the request payload
2. Calls the Bedrock Runtime converse() API
3. Extracts the text response from the model
4. Returns both the text output and full response object

In [8]:
def amazon_bedrock_nova_converse(model, messages, temperature, max_tokens, top_p = 0.9, system = None, reasoning_effort = None, tools = None, bedrock_rt_client = None ):  
    if not bedrock_rt_client:
        bedrock_rt_client = create_boto3_client()

    try:
        request = construct_converse_payload(model, messages, temperature, max_tokens, top_p, system, reasoning_effort, tools)

        model_response = bedrock_rt_client.converse(**request)
        
        output = model_response["output"]["message"]["content"][-1]["text"]
        return output, model_response
        
    except Exception as e:
        print(type(e), e)
        return str(e), e

    

### Invoke Amazon Nova with Tool Interpreter with the Converse API

This example demonstrates the basic workflow of using Code Interpreter with the converse API. To give Amazon Nova access to the Code Interpreter you specify the `systemTool` parameter in the tools config. 

In [9]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Print hello world using the code interpreter"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

#### Understanding the Response

The model response contains multiple layers of information. Let's examine each part.
Here is the final output from Amazon Nova:

In [10]:
print("Final response from Amazon Nova:")
print(output)

Final response from Amazon Nova:
The code interpreter has printed:

```
Hello, World!
```

This is the classic "Hello World" greeting. If you'd like to print additional messages or perform other computations, just let me know!


In addition to the final output, you can examine the model response to understand
what code was executed in the Code Interpreter. This is valuable for:
- Debugging: See exactly what code was generated
- Learning: Understand how the model approaches problems
- Auditing: Track what code is being executed in your application

In [10]:
content = model_response['output']['message']['content']

Extract tool use and tool result from the response content:

In [11]:
tool_use = next((trace for trace in content if "toolUse" in trace), None)
tool_result = next((trace for trace in content if "toolResult" in trace), None)

The `toolUse` block shows what code Amazon Nova generated and sent to the Code Interpreter. The `toolUse` > `input` > `code` property contains the code that got executed in the Code Interpreter.

In [12]:
JSON(tool_use, expanded=True)

<IPython.core.display.JSON object>

The output of the Code Interpreter tool contains the logs from `stdOut` in the Code Interpreter environment and also any errors that occured in the Code Interpreter environment.

In [13]:
JSON(tool_result, expanded=True)

<IPython.core.display.JSON object>

### Converse API with Streaming

Code Interpreter also works with streaming the response. Streaming allows you to receive the model's response incrementally as it's generated, rather than waiting for the complete response.

The `process_stream` helper function handles receiving the stream of output blocks.

In [18]:
def process_stream(stream_response):
    is_reasoning = False
    is_text = False
    is_toolUse = False
    is_tool_result = False
    model_output = ""  # Initialize text collector
    
    for message in stream_response["stream"]:
        if "contentBlockStart" in message:
            if "toolUse" in message["contentBlockStart"]["start"]:
                is_toolUse = True
                is_text = False
                is_reasoning = False
                print("\n\n----MODEL RESPONSE: TOOL USE-----\n")
                tool_use = message["contentBlockStart"]["start"]["toolUse"]
                print(tool_use["name"])

        if "contentBlockDelta" in message:
            delta = message["contentBlockDelta"]["delta"]

            # Check for regular text
            if "text" in delta:
                if not is_text:
                    print("\n\n----MODEL RESPONSE: TEXT-----\n")
                    is_text = True
                    is_reasoning = False
                    is_toolUse = False
                text_chunk = delta["text"]
                model_output += text_chunk
                print(text_chunk, end="", flush=True)

            # Check for reasoning content
            elif "reasoningContent" in delta and "text" in delta["reasoningContent"]:
                if not is_reasoning:
                    print("\n\n----MODEL RESPONSE: REASONING-----\n")
                    is_text = False
                    is_reasoning = True
                    is_toolUse = False
                print(delta["reasoningContent"]["text"], end="", flush=True)

            elif "toolUse" in delta:
                print(delta)
            elif "toolResult" in delta:
                if not is_tool_result:
                    print("\n\n----MODEL RESPONSE: TOOL RESULT-----\n")
                    is_tool_result = True
                    is_toolUse = False
                print(delta)
        elif "messageStop" in message:
            print("\n\n----MODEL RESPONSE: messageStop-----\n")
            print(json.dumps(message, indent=4))
        elif "metadata" in message:
            print("\n\n----MODEL RESPONSE: metadata-----\n")
            print(json.dumps(message, indent=4))
    
    return model_output


Similar to the non-streaming version, but calls `converse_stream` instead. The function returns a stream object that must be iterated to receive events.

In [16]:
def amazon_bedrock_nova_converse_streaming(model, messages, temperature, max_tokens, top_p = 0.9, system = None, reasoning_effort = None, tools = None, bedrock_rt_client = None ):  
    if not bedrock_rt_client:
        bedrock_rt_client = create_boto3_client()

    try:
        request = construct_converse_payload(model, messages, temperature, max_tokens, top_p, system, reasoning_effort, tools)

        model_response = bedrock_rt_client.converse_stream(**request)
        
        return model_response
        
    except Exception as e:
        print(type(e), e)
        return e

    

### Invoke Amazon Nova with Web Grounding using the Converse Streaming API 

Next you will invoke the Converse Streaming API with Web Grounding system tool configured.

Note: In this example we are showing the Web Grounding system tool with the Converse Streaming API. However, the system tool is not specific to which API you use. You can also use the converse streaming API with Code Interpreter and vice versa the non-streamin converse API works with Web Grounding. Later you will also see how to use both system tools together. Which API to use depends on your application. For a chabot interface the converse streaming API is more suitable if you want to stream the response to the user.

In [20]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "What is the weather in Seattle today?"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_grounding"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

response = amazon_bedrock_nova_converse_streaming(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
model_output = process_stream(response)



----MODEL RESPONSE: TOOL USE-----

nova_grounding
{'toolUse': {'input': '{"query":"What is the weather in Seattle today?"}'}}


----MODEL RESPONSE: TOOL RESULT-----

{'toolResult': [{'text': '[HIDDEN]'}]}


----MODEL RESPONSE: TEXT-----

The current weather in Seattle is 44°F with showers in the vicinity. The temperature feels like 43°F, with 85% humidity and light winds from the southeast at 3 mph. 

According to the hourly forecast, the temperature will remain around 44°F for the next few hours, with a chance of precipitation varying between 6% and 38%. The wind will continue to be light, primarily from the southeast or south/southeast direction, ranging from 3 to 5 mph. 

The 10-day forecast shows temperatures ranging from the high 40s to low 50s°F, with varying chances of precipitation from 24% to 94%.

----MODEL RESPONSE: messageStop-----

{
    "messageStop": {
        "stopReason": "end_turn"
    }
}


----MODEL RESPONSE: metadata-----

{
    "metadata": {
        "usage": {
 

In [19]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Calculate 7^6 using the code interpreter"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

response = amazon_bedrock_nova_converse_streaming(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
model_output = process_stream(response)



----MODEL RESPONSE: TOOL USE-----

nova_code_interpreter
{'toolUse': {'input': '{"snippet":"# Calculate 7^6\\nresult = 7 ** 6\\nprint(result)"}'}}


----MODEL RESPONSE: TOOL RESULT-----

{'toolResult': [{'SDK_UNKNOWN_MEMBER': {'name': 'json'}}]}


----MODEL RESPONSE: TEXT-----

The result of \(7^6\) is **117,649**.

----MODEL RESPONSE: messageStop-----

{
    "messageStop": {
        "stopReason": "end_turn"
    }
}


----MODEL RESPONSE: metadata-----

{
    "metadata": {
        "usage": {
            "inputTokens": 957,
            "outputTokens": 59,
            "totalTokens": 1016
        },
        "metrics": {
            "latencyMs": 2048
        }
    }
}


The final response from Amazon Nova is:

In [18]:
Markdown(model_output)

The result of \(7^6\) is **117,649**.

## 3. Sample Use Cases that require Code execution

This section demonstrates real-world scenarios where Code Interpreter excels. These examples showcase the power of combining LLM reasoning with code execution for tasks that require precise computation.

#### Solving Polynomial Equations

Polynomial equations often have complex or multiple roots that are difficult to find even for reasoning models. Code Interpreter can use numerical methods (like NumPy's roots function) to find all solutions, including complex roots.

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Use the code interpreter to solve for all roots of the polynomial equation: z^6 + z^4 + z^3 + z^2 + 1 = 0"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 10000
reasoning_effort = None

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

Here is the solution for the polynomial:

In [ ]:
Markdown(output)

#### Statistical Hypothesis Testing

Statistical analysis is a perfect use case for Code Interpreter because it requires:
- Precise mathematical calculations
- Standard statistical libraries (scipy, numpy)
- Amazon Nova's capability to interpret the results in context

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": """I have two datasets of student test scores. Perform a t-test to determine if there's a statistically significant difference between them at α=0.05 confidence level.

Group A: [78, 85, 92, 88, 79, 91, 84, 87, 90, 82]
Group B: [72, 75, 81, 69, 74, 78, 71, 76, 73, 70]
"""
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 10000
reasoning_effort = None

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

Here is the output of the statistical analysis that Amazon Nova conducted using the code interpreter:

In [ ]:
print("\nStatistical analysis results:")
Markdown(output)

#### Multi-Turn Conversations with Code Interpreter

This section demonstrates a realistic business scenario where a coffee shop owner uses data analysis to reduce waste and make informed decisions.

In this example, we'll simulate a conversation with Maria, a coffee shop owner who wants to understand and reduce her daily pastry waste. The conversation progresses from initial problem identification to data analysis and decision-making.


In [33]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "I'm throwing away so many pastries at the end of each day. It breaks my heart and wastes money. What can I do about this?"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

system = [{ "text": "You are a helpful assistant. Keep your answers short." }]

temperature = 0.7
top_p = 0.9
max_tokens = 1000

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, system=system, tools=tools, bedrock_rt_client=bedrock_client)


In [34]:
print("Turn 1 - Initial consultation:")
print(output)

Turn 1 - Initial consultation:
It's unfortunate to see pastries go to waste! There are many ways to minimize waste and ensure your pastries are enjoyed. Here are some suggestions to help you reduce pastry waste at the end of each day:

---

### **1. Accurate Forecasting & Ordering**
- **Track Sales Patterns:** Keep a record of how many pastries are sold each day for a week or two. This will help you identify trends and understand which items are more popular and which are not.
- **Data-Driven Ordering:** Use this data to order the right quantity of pastries. Adjust based on special events, holidays, or seasonal changes.

---

### **2. Optimize Display and Freshness**
- **First In, First Out (FIFO):** Ensure older pastries are displayed at the front and newer ones placed at the back to encourage selling the older stock first.
- **Proper Storage:** If space allows, keep some pastries in a proper storage area (like refrigerated or frozen) to extend their shelf life for a day or two.

---


In [26]:

messages.append({
    "role": "assistant",
    "content": [{"text": output}]
})

messages.append({
    "role": "user",
    "content": [{
        "text": "I'd say 8-12 pastries per day, mostly croissants and muffins. I order the same amount every week from my supplier—4 dozen croissants, 3 dozen muffins, 2 dozen danishes. It's just what I've always done."
    }]
})

In [27]:
output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, tools=tools, bedrock_rt_client=bedrock_client)

In [28]:
print("Turn 2 - Understanding the pattern:")
print(output)

Turn 2 - Understanding the pattern:
### Weekly Waste Calculation  
If you throw away **8–12 pastries per day** (average **10**), you’re discarding about **70 pastries per week**. That’s a significant loss of both food and money.  

Let’s break down your current order vs. actual need.  

| Pastry Type | Current Weekly Order | Approx. Weekly Sales (based on 10 wasted/day) |
|-------------|----------------------|-----------------------------------------------|
| Croissants | 4 dozen = **48**     | ~ (48 – 10) = **38** sold                     |
| Muffins     | 3 dozen = **36**     | ~ (36 – 10) = **26** sold                     |
| Danishes    | 2 dozen = **24**     | ~ (24 – 10) = **14** sold                     |

> **Note**: These are rough estimates because you waste 10 pastries total per day, not per type. Adjust based on your actual sales data.  

---

## Action Plan to Reduce Waste  

### 1. Track Daily Sales for 2 Weeks  
Start a **simple spreadsheet** (Google Sheets or Excel). Co

In [29]:
messages.append({
    "role": "assistant",
    "content": [{"text": output}]
})

messages.append({
    "role": "user",
    "content": [{
        "text": """Okay, I tracked it for a week. Here's what I threw away each day:

- Monday: 6 croissants, 4 muffins
- Tuesday: 5 croissants, 3 muffins
- Wednesday: 8 croissants, 5 muffins, 2 danishes
- Thursday: 4 croissants, 2 muffins, 1 danish
- Friday: 2 croissants, 1 muffin
- Saturday: 1 croissant, 0 muffins
- Sunday: 3 croissants, 2 muffins

Each croissant costs me $1.50, muffins $1.20, and danishes $1.80. How much am I losing?"""
    }]
})

In [30]:
output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, tools=tools, bedrock_rt_client=bedrock_client)

In [31]:
print("Turn 3 - Waste analysis with calculations:")
print(output)

Turn 3 - Waste analysis with calculations:
### Weekly Waste Cost  

You threw away the following pastries and their dollar value each day:

| Day       | Croissants | Muffins | Danishes | Waste Cost |
|-----------|------------|---------|----------|------------|
| Monday    | 6          | 4       | 0        | $13.80 |
| Tuesday   | 5          | 3       | 0        | $11.10 |
| Wednesday | 8          | 5       | 2        | $21.60 |
| Thursday  | 4          | 2       | 1        | $10.20 |
| Friday    | 2          | 1       | 0        | $4.20  |
| Saturday  | 1          | 0       | 0        | $1.50  |
| Sunday    | 3          | 2       | 0        | $6.90  |
| **Total** |            |         |          | **$69.30** |

You’re losing **$69.30 per week** from pastry waste.


------

Let's examine the code that was executed:

In [32]:
content = model_response['output']['message']['content']
tool_use = next((trace for trace in content if "toolUse" in trace), None)
if tool_use and "input" in tool_use["toolUse"]:
    print("Code executed by Amazon Nova:")
    print(tool_use["toolUse"]["input"]["code"])

Code executed by Amazon Nova:
"""Calculate weekly waste cost from the given data."""
# Waste per day
waste = {
    "Monday": {"croissants": 6, "muffins": 4, "danishes": 0},
    "Tuesday": {"croissants": 5, "muffins": 3, "danishes": 0},
    "Wednesday": {"croissants": 8, "muffins": 5, "danishes": 2},
    "Thursday": {"croissants": 4, "muffins": 2, "danishes": 1},
    "Friday": {"croissants": 2, "muffins": 1, "danishes": 0},
    "Saturday": {"croissants": 1, "muffins": 0, "danishes": 0},
    "Sunday": {"croissants": 3, "muffins": 2, "danishes": 0}
}

cost = {"croissants": 1.50, "muffins": 1.20, "danishes": 1.80}
total_loss = 0
details = {}
for day, items in waste.items():
    loss = sum(items[k] * cost[k] for k in items)
    details[day] = loss
    total_loss += loss

total_loss, details


## 4. Best Practices for Code Intepreter

TODO

## 5. Next Steps

TODO